In [2]:
import numpy as np

hidden_layers = int(input("Брой неврони от скрития слой"))
learn_repeat = int(input("Брой повторения на обучаването"))
data = np.array([[0,0],[0,1],[1,0],[1,1]])
y_and = np.array([0,0,0,1])
y_or = np.array([0,1,1,1])
y_xor = np.array([0,1,1,0])

def sigmoid(z):
    return 1/(1 + np.exp(-z))

weights_hidden = np.random.uniform(-0.05, 0.05, (hidden_layers, data.shape[1]))
weights_output = np.random.uniform(-0.05, 0.05, (1, hidden_layers))

for i in range(learn_repeat):
    hidden = np.dot(weights_hidden, data.T)
    # print(hidden.shape)
    hidden = sigmoid(hidden)
    output = np.dot(weights_output, hidden)
    # print(output.shape)
    print(output)
    weights_output = weights_output + 0.1 * output*(1-output)*(y_xor - output)
    weights_hidden = weights_hidden + 0.1 * np.dot((hidden*(1-hidden)).T, (weights_output.T * (y_xor - output))).T
    

KeyboardInterrupt: 

In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        # Инициализация на теглата
        self.weights_input_hidden = np.random.uniform(-0.05, 0.05, (input_size, hidden_size))
        self.weights_hidden_output = np.random.uniform(-0.05, 0.05, (hidden_size, output_size))
        
        self.bias_hidden = np.zeros(hidden_size)
        self.bias_output = np.zeros(output_size)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, inputs):
        # Изчисляване на активациите
        self.hidden_input = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)
        
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self.sigmoid(self.final_input)

        return self.final_output

    def backward(self, inputs, targets):
        # Изчисляване на грешките
        output_errors = targets - self.final_output
        output_delta = output_errors * self.sigmoid_derivative(self.final_output)

        hidden_errors = np.dot(output_delta, self.weights_hidden_output.T)
        hidden_delta = hidden_errors * self.sigmoid_derivative(self.hidden_output)

        # Обновяване на теглата
        self.weights_hidden_output += self.learning_rate * np.dot(self.hidden_output.T, output_delta)
        self.bias_output += self.learning_rate * output_delta.sum(axis=0)

        self.weights_input_hidden += self.learning_rate * np.dot(inputs.T, hidden_delta)
        self.bias_hidden += self.learning_rate * hidden_delta.sum(axis=0)

    def train(self, inputs, targets, epochs):
        for epoch in range(epochs):
            outputs = self.forward(inputs)
            self.backward(inputs, targets)
            if (epoch + 1) % 10 == 0:
                loss = np.mean((targets - outputs) ** 2)
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

# Брой входни, скрити и изходни неврони
input_size = 2  # Примерен брой атрибути
hidden_size = 5 # Параметър
output_size = 1 # Примерен брой класове
learning_rate = 0.1

# Примерни данни
inputs = np.array([[0, 0], [ 0, 1], [1, 0], [1, 1]])
targets = np.array([ [0], [1], [1], [0]])

# Създаване и обучение на мрежата
nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
nn.train(inputs, targets, epochs=1000)

# Показване на резултатите след трениране
print("\nResults after training:")
for i, input_sample in enumerate(inputs):
    prediction = nn.forward(input_sample)
    print(f"Input: {input_sample}, Predicted: {prediction}, Target: {targets[i]}")



In [4]:
def crossEntropyS(X, Y, w, b):
	# X.shape = (S,N) Y.shape = (S), W.shape = (N) 
    v = sigmoid(np.dot(X,w)+b)
    p = (1-Y) + (2*Y-1)*v
    ce = -np.mean(np.log(p))
    return ce

def gradCrossEntropyS(X,Y,w,b):
    g = Y - sigmoid(np.dot(X,w)+b)
    db = -np.mean(g)
    dw = -np.mean( g[:,np.newaxis] * X,axis=0)
    return dw, db

In [ ]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# hidden_layers = int(input("Брой неврони от скрития слой"))
# learn_repeat = int(input("Брой повторения на обучаването"))
k = 5
learn_repeat = 100
learning_rate = 0.5
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y_and = np.array([0,0,0,1])
y_or = np.array([0,1,1,1])
y_xor = np.array([0,1,1,0])

y = y_or

Wh = np.random.uniform(-0.1, 0.1, (X.shape[1], k))
bh = np.random.uniform(-0.1, 0.1, (k,))

Wo = np.random.uniform(-0.1, 0.1, (k, ))
bo = np.random.uniform(-0.1, 0.1)

for i in range(learn_repeat):
    q = np.dot(X, Wh) + bh
    r = sigmoid(q)
    s = np.dot(r, Wo) + bo
    sc = sigmoid(s)
    p = (1-y) + (2*y-1)*sc
    error = -np.mean(np.log(p))
    # print(p)
    # print(error)
    
    dWo, dbo = gradCrossEntropyS(r,y,Wo,bo)
    ds = -np.mean(y - sc)
    dt = ds
    dr = np.outer(np.ones(k), ds).T * np.outer(np.ones(X.shape[0]), Wo)
    dq = dr * sigmoid_derivative(q)
    dbh = dq
    dWh = X.T @ dq
    
    Wh = Wh - learning_rate * dWh
    Wo = Wo - learning_rate * dWo
    bh = bh - learning_rate * dbh
    bo = bo - learning_rate * dbo
print(sc)
print(error)

[0.74791612 0.74870214 0.74875909 0.74954269]
0.5612593570070752


In [28]:
import numpy as np

# Define tensors with hardcoded values
A = np.array([
    [
        [[1000, 0, 0], [2000, 0, 0]],
        [[0, 1000, 0], [0, 2000, 0]],
        [[0, 0, 1000], [0, 0, 2000]]
    ],
    [
        [[100, 0, 0], [200, 0, 0]],
        [[0, 100, 0], [0, 200, 0]],
        [[0, 0, 100], [0, 0, 200]]
    ],
    [
        [[10, 0, 0], [30, 0, 0]],
        [[0, 10, 0], [0, 30, 0]],
        [[0, 0, 10], [0, 0, 30]]
    ]
])  # Shape (3, 3, 2, 3)

B = np.array([
    [1, 2, 3],
    [1, 1, 1],
    [-1, -1, -1]
])  # Shape (3, 3)

# Reshape B to be broadcastable with A
B_broadcasted = B[:, :, np.newaxis, np.newaxis]  # Shape (3, 3, 1)

# Perform elementwise multiplication and sum along axis 0
result = (A * B_broadcasted).sum(axis=(0,1))

print(result.shape)  # Should output (2, 3)
print(result)


(2, 3)
[[1090 2090 3090]
 [2170 4170 6170]]
